[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ArturGogiyan/NLP_research/blob/master/XGBoost.ipynb)

# Configure environment

Following section required only if you're running this code in google collab, so if you need uncomment it and run

In [1]:
# !git clone https://github.com/ArturGogiyan/NLP_research.git
# %cd NLP_research
# !pip install -r './requirements.txt'

# Dataset loading

In [2]:
import pandas as pd
import dvc.api

!pip install 'dvc[gdrive]'

You should consider upgrading via the '/home/basicec/PycharmProjects/NLP_research2/venv/bin/python -m pip install --upgrade pip' command.


In [3]:
def read_dataset():
    with dvc.api.open('dataset/post_processed.csv', repo='https://github.com/ArturGogiyan/NLP_research') as fd:
        return pd.read_csv(fd)

In [5]:
df = read_dataset()

print("Shape of data=>", df.shape)

/home/basicec/PycharmProjects/NLP_research2/venv/lib/python3.8/site-packages/oauth2client/_helpers.py:255: UserWarning: Cannot access /tmp/tmp54tmtzwxdvc-clone/.dvc/tmp/gdrive-user-credentials.json: No such file or directory
  warnings.warn(_MISSING_FILE_MESSAGE.format(filename))


Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?client_id=710796635688-iivsgbgsb6uv1fap6635dhvuei09o66c.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.appdata&access_type=offline&response_type=code&approval_prompt=force

Authentication successful.
Shape of data=> (5000, 17)


# Non-neural network processing

## Preparations

Resources needed:

In [6]:
import xgboost as xgb
import numpy as np
from sklearn.metrics import precision_score, recall_score, accuracy_score

from common import bagofwords
from common import utils

## XGBoost

In [7]:
def xgb_test(test_percentage):
    texts_train, texts_test, y_train, y_test = utils.get_train_and_test(df, "All", test_percentage)
    if len(texts_train) == 0:
        print("Class entities amount is too small! Ignoring...")
        return [], [], []

    bow = bagofwords.BagOfWords(texts_train)
    X_train_bow = np.stack(list(map(bow.text_to_bow, texts_train)))
    X_test_bow = np.stack(list(map(bow.text_to_bow, texts_test)))
    D_train = xgb.DMatrix(X_train_bow, label=y_train)
    D_test = xgb.DMatrix(X_test_bow, label=y_test)

    param = {
        'eta': 0.3,
        'max_depth': 3,
        'objective': 'multi:softprob',
        'num_class': len(list(df.columns.values))}

    steps = 20

    xgb_model = xgb.train(param, D_train, steps)

    preds = xgb_model.predict(D_test)
    best_preds = np.asarray([np.argmax(line) for line in preds])

    print("Precision = {}".format(precision_score(y_test, best_preds, average='macro')))
    print("Recall = {}".format(recall_score(y_test, best_preds, average='macro')))
    print("Accuracy = {}".format(accuracy_score(y_test, best_preds)))
    return xgb_model, bow


xgb_model, bow = xgb_test(50)
xgb_model.save_model('./model/xgb.json')
bow.save('./model/xgb_bow.json')

[20:18:09] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/home/basicec/PycharmProjects/NLP_research2/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Precision = 0.4597434542682396
Recall = 0.37303161097325616
Accuracy = 0.8412
